# Computational Physics 2022 - E3 Monte Carlo Methods

$\providecommand\d[1]{\;\mathrm{d}#1\@ifnextchar\d{\!}{}}$

## Task 1:

$$
\text{Evaluate} \quad I = \int_0^1 f(x) \d x \quad \text{with} \quad f(x) = x(1-x)
$$
Analytic solution:
$$
\begin{align}
I &= \int_0^1 x(1-x) \d x = \int_0^1 x \d x - \int_0^1 x^2 \d x \\
&= \left[ \frac{1}{2} x^2 \right]_0^1 - \left[ \frac{1}{3} x^3 \right]_0^1 
= \frac{1}{2} - \frac{1}{3} \\ &= \frac{1}{6}
\end{align}
$$

Monte Carlo method:
- Generate $N$ samples $x_i$ from a uniform distribution between the lower and upper integration boundary.
- Evaluate the function at all N points $\Rightarrow f_i = f(x_i)$
- Approximate the integral as the mean $I \approx \frac{1}{N} \sum_i f_i$
- The variance in this approximation is $\sigma^2 \approx \frac{1}{N} \left( \frac{1}{N} \sum_i f_i^2 - \left(\frac{1}{N} \sum_i f_i \right)^2 \right)$

<img src="plots/E3_1.png" width="500" height="400"> 

## Task 2:

Monte Carlo Integration with importance sampling:
- Choose a probability distribution $p(x)$ that has a similar shape to $f(x)$
- Generate $N$ samples $x_i$ from $p(x)$ using the transformation method:
    - Generate samples $u_i$ from a uniform distribution between 0 and 1
    - Transform them using $x_i = F^{-1}(u_i)$ where $F(x) = \int_{-\infty}^x p(x') \d x'$
- Evaluate and weight the function at all N points $\Rightarrow g_i = g(x_i) = f(x_i)/p(x_i)$
- Approximate the integral as the mean $I \approx \frac{1}{N} \sum_i g_i$
- The variance of this approximation is $\sigma^2 \approx \frac{1}{N} \left( \frac{1}{N} \sum_i g_i^2 - \left(\frac{1}{N} \sum_i g_i \right)^2 \right)$

Here, we use $p(x) ~ \sin(\pi x)$. Since we need a normalized pdf with $\int_{-\infty}^\infty p(x) \d x = 1$, we use
$$
p(x) = c \cdot \sin(\pi x) \cdot \mathbb{1}_{[0,1]} 
$$
with
$$
    \int_{-\infty}^\infty p(x) \d x = c \int_0^1 \sin(\pi x) \d x = c \left[ -\frac{1}{\pi} \cos(\pi x) \right]_0^1 = 1 \\
    \Leftrightarrow c = \frac{\pi}{2}
$$
Let's calculate the cdf and its inverse:
$$
F(x) = \int_{-\infty}^x p(x') \d x' = \frac{\pi}{2} \int_0^x \sin(\pi x') \d x' = \frac{\pi}{2} \left[ -\frac{1}{\pi} \cos(\pi x) \right]_0^1 \\
F(x) = \frac{1}{2} \left( 1 - \cos(\pi x) \right) \\
\Leftrightarrow \cos(\pi x) = 1 - 2F(x) \\
\Leftrightarrow x(F) = \frac{1}{\pi} \arccos(1 - 2F) = F^{-1}(y)
$$
And the function weighting:
$$
g(x) = \frac{f(x)}{p(x)} = \frac{x(1-x)}{\frac{\pi}{2} \sin(\pi x)}
$$

Results:  
<img src="plots/E3_2.png" width="500" height="400">  
<img src="plots/E3_2_sigma.png" width="500" height="400">  
<img src="plots/E3_2_x.png" width="500" height="400">  

## Task 3:
Compute the following integral using monte carlo integration with importance sampling:
$$
I = \pi^{-3/2} \int_{-\infty}^{+\infty} \int_{-\infty}^{+\infty} \int_{-\infty}^{+\infty} 
x^2 + x^2 y^2 + x^2 y^2 z^2 \exp \left[ - \left(x^2+y^2+z^2\right) \right] \d x \d y \d z
$$
for the importance sampling use the weightfunction
$$
p(x,y,z) = \pi^{-3/2} \exp \left[ - \left(x^2+y^2+z^2\right) \right]
$$
also define
$$
g(x,y,z) = x^2 + x^2 y^2 + x^2 y^2 z^2 = x^2 (1 + y^2 (1 + z^2)) \\
\Rightarrow 
I = \underset{\mathbb{R^3}}{\int \int \int} g(x,y,z) p(x,y,z) \d x \d y \d z
$$
which is precisely the integral used in importance sampling where we get the integral as
$$
I \approx \frac{1}{N} \sum_i g(x_i, y_i, z_i)
$$
if $x_i, y_i, z_i$ are distributed as $p(x, y, z)$.

To generate samples from $p$, use the Metropolis algorithm:
- start at any point $r_0 = (x_0, y_0, z_0)$
- compute $p(x_0, y_0, z_0)$
- for iteration $i+1$ choose a next "trial" point $\tilde{r}_{i+1}$ on the basis of $r_{i}$
    - in our case $\tilde{x}_{i+1} = x_i + \Delta (u - 0.5)$ (the same for $y,z$)
    - $u$ is a uniform random number between 0 and 1
    - $\Delta$ is a parameter to adjust (we chose $\Delta = 2$)
- if $p(\tilde{r}_{i+1})/p(r_i) >= u$, where us is again a standard uniform random number, 
    - then accept $r_{i+1} = \tilde{r}_{i+1}$
    - else stay at $r_{i+1} = r_i$
- throw away the first samples, because this algorithm needs a burn in phase
- the samples $r_{n_\text{burn}+1}, ..., r_{n_\text{burn}+n_\text{steps}}$ are distributed according to $p$

Results:
$$
I \approx 0.8856953247
$$
according to [WolframAlpha](https://www.wolframalpha.com/input?key=&i=pi%5E%28-3%2F2%29+%E2%88%AB_%28-%E2%88%9E%29%5E%28+%E2%88%9E%29+%E2%88%AB_%28-%E2%88%9E%29%5E%28+%E2%88%9E%29+%E2%88%AB_%28-%E2%88%9E%29%5E%28+%E2%88%9E%29+%28x%5E2*%281+%2B+y%5E2*%281+%2B+z%5E2%29%29+*+exp%28-x%5E2-y%5E2-z%5E2%29%29+dx+dy+dz): 
$$
I = \frac{7}{8} = 0.875
$$
We chose $\Delta = 2.0$, because with this we get an acceptance rate of $0.50325$.

Sample distribution (100000 samples):  
<img src="plots/E3_3_samples.png" width="500" height="400"> 
